In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
from mlxtend.classifier import StackingClassifier

In [29]:
sub1=pd.read_csv('XGB_21_NOV_78.131_.0025eta_2044iter.csv')
sub2=pd.read_csv('CNN.csv')

In [18]:
li=[]
li=list(sub2['target'].values)

In [30]:
sub1

,connection_id,target
0,cxcon_2,1
1,cxcon_5,0
2,cxcon_8,0
3,cxcon_11,0
4,cxcon_14,0
5,cxcon_17,0
6,cxcon_20,0
7,cxcon_23,1
8,cxcon_26,0
9,cxcon_29,0


In [28]:
sub1.target.value_counts(normalize=True)

0    0.583957
2    0.217676
1    0.198367
Name: target, dtype: float64

In [6]:
pd.concat([sub1['target'],sub2['target']],1).corr()

,target,target
target,1.000000,0.254913
target,0.254913,1.000000


In [ ]:
class Ensemble(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models

    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)

        folds = list(StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=2016).split(X, y))

        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):

            S_test_i = np.zeros((T.shape[0], self.n_splits))

            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
#                y_holdout = y[test_idx]

                print ("Fit %s fold %d" % (str(clf).split('(')[0], j+1))
                clf.fit(X_train, y_train)
#                cross_score = cross_val_score(clf, X_train, y_train, cv=3, scoring='roc_auc')
#                print("    cross_score: %.5f" % (cross_score.mean()))
                #for probabilty prediction
                # y_pred = clf.predict_proba(X_holdout)[:,1]                
                y_pred = clf.predict(X_holdout)[:,1]                

                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict(T)[:,1]
            S_test[:, i] = S_test_i.mean(axis=1)

        results = cross_val_score(self.stacker, S_train, y, cv=3, scoring='roc_auc')
        print("Stacker score: %.5f" % (results.mean()))

        self.stacker.fit(S_train, y)
        res = self.stacker.predict_proba(S_test)[:,1]
        return res


In [ ]:
log_model = LogisticRegression()


        
stack = Ensemble(n_splits=3,
        stacker = log_model,
        base_models = (lgb_model, lgb_model2, lgb_model3))        
        
y_pred = stack.fit_predict(train, target_train, test)